In [1]:
import threading

import findspark
findspark.init()

import pyspark

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Partition Run").getOrCreate()

In [2]:
class Sentiment:
    Positive = "Positive"
    Negative = "Negative"
    Neutral = "Neutral"

class Review:
    def __init__(self,text,score):
        self.text = text
        self.score = score
        self.sentiment = self.getSentiment()
        
    def getSentiment(self):
        if (self.score <= 2):
            return Sentiment.Negative
        elif self.score == 3:
            return Sentiment.Neutral
        else:
            return Sentiment.Positive
        



In [3]:
f = spark.read.json('/home/shivam/intern/rev.json')
f.rdd.getNumPartitions()

8

In [5]:
from pyspark.sql.functions import row_number,col,lit
from pyspark.sql.window import Window

In [10]:
f.count()

571337

In [6]:
df = f.select("overall","reviewText")
df = df.withColumn("overint",df["overall"].cast("Int"))
df2 = df.select("overint","reviewText").orderBy("overint")

In [7]:
w = Window().partitionBy(lit('a')).orderBy(lit('a'))

df1 = df2.withColumn("row_num", row_number().over(w))

In [13]:
df2.printSchema()

root
 |-- overint: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- row_num: integer (nullable = true)



In [62]:
dat = df1.filter(col("row_num").between(457068,571337)).select("overint","reviewText")

In [63]:
data = dat.repartition(1)

In [64]:
data.write.mode("overwrite").json("/home/shivam/intern/p1")

In [65]:
data.rdd.getNumPartitions()

1

In [66]:
import json

part0 = "/home/shivam/intern/p1/part-00000-97cad775-1303-45f5-a344-b9b88e4909e7-c000.json"

reviews1 = []
with open(part0) as p0:
    for line in p0:
        review = json.loads(line)
        reviews1.append(Review(review['reviewText'],review['overint']))

In [3]:
i = 0
n = 5
while n>0:
    count = 0
    while count < 114267:
        count = count +1 

    print(count)
    print(n)
    n = n-1
    i = i+1

SyntaxError: 'return' outside function (<ipython-input-3-07bca4cccd09>, line 8)

In [12]:
import json

part1 = "/home/shivam/intern/parts/part-00001-a2397ecf-c83f-49c7-8ad2-d9e042ffeca9-c000.json"

reviews2 = []
with open(part1) as p1:
    for line in p1:
        review = json.loads(line)
        reviews2.append(Review(review['reviewText'],review['overint']))

In [13]:
import json

part2 = "/home/shivam/intern/parts/part-00002-a2397ecf-c83f-49c7-8ad2-d9e042ffeca9-c000.json"

reviews3 = []
with open(part2) as p2:
    for line in p2:
        review = json.loads(line)
        reviews3.append(Review(review['reviewText'],review['overint']))

In [14]:
import json

part3 = "/home/shivam/intern/parts/part-00003-a2397ecf-c83f-49c7-8ad2-d9e042ffeca9-c000.json"

reviews4 = []
with open(part3) as p3:
    for line in p3:
        review = json.loads(line)
        reviews4.append(Review(review['reviewText'],review['overint']))

In [11]:
import json

part4 = "/home/shivam/intern/parts/part-00004-a2397ecf-c83f-49c7-8ad2-d9e042ffeca9-c000.json"

reviews5 = []
with open(part4) as p4:
    for line in p4:
        review = json.loads(line)
        reviews5.append(Review(review['reviewText'],review['overint']))

In [67]:
import time
start_time = time.time()

from sklearn.model_selection import train_test_split

training,test = train_test_split(reviews1,test_size = 0.25,random_state = 40)
train_x = [x.text for x in training]
train_y = [x.sentiment for x in training]

test_x = [x.text for x in test]
test_y = [x.sentiment for x in test]

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

In [69]:
from sklearn.ensemble import RandomForestClassifier
clf_dec = RandomForestClassifier()
clf_dec.fit(train_x_vectors,train_y)
x = clf_dec.predict(test_x_vectors)

#from sklearn.linear_model import LogisticRegression
#clf_lg = LogisticRegression()
#clf_lg.fit(train_x_vectors,train_y)
#clf_lg.predict(test_x_vectors)

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

--- 7.0134172439575195 seconds ---


In [19]:
reviews1[114266].text

"Really comfortable. I ordered a second one. You could use them under jeans in cold weather.  I am 5'9 and they are nice and long. might be too long for a shorter person."

In [20]:
train_x_vectors.shape
len(train_y)
test_x_vectors.shape

(28567, 56023)

In [21]:
train_x_vectors.shape

(85700, 56023)